## Import libraries

In [2]:
import os
import numpy as np
import json
from qdrant_client import QdrantClient
from qdrant_client.http import models
from app.utils import get_video_pack_files, get_keyframe_data
from app.PATH import CLIP_FEATURES_PATH, MAP_KEYFRAMES_PATH

2025-08-30 16:42:58.576 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


## Create Qdrant client

In [10]:
client = QdrantClient(host="localhost", port=6333)
client

## Constant

In [11]:
VIDEO_PACK_LIST = [pack for pack in range(21, 31)]
COLLECTION_NAME = "my_collection"

## Create colletion

In [20]:
if COLLECTION_NAME not in [c.name for c in client.get_collections().collections]:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(size=512, distance=models.Distance.COSINE),
        optimizers_config=models.OptimizersConfigDiff(
            indexing_threshold=0,
        ),
    )
    print(f"Collection '{COLLECTION_NAME}' created.")
else:
    print(f"Collection '{COLLECTION_NAME}' already exists.")

Collection 'my_collection' created.


## Upsert data automatically

In [21]:
start_id = 0
process_file = 0
if not os.path.exists(CLIP_FEATURES_PATH):
    print("The folder does not exist")
else:
    start_id = 0
    print(os.listdir(CLIP_FEATURES_PATH)[:5])
    for pack in VIDEO_PACK_LIST:
        video_pack = f"L{pack}"
        print(f"Process {video_pack}")
        files_per_video_pack = get_video_pack_files(CLIP_FEATURES_PATH, video_pack)
        # A list of .npy files, each file contains a list of 512 features vectors, 
        # each vector is embedded from a keyframe of the video that has the same name as the file

        if len(files_per_video_pack) == 0:
            print(f"No files found for video pack {video_pack}")
            continue

        for file in files_per_video_pack:
            # Process each video features in the pack
            feature = np.load(os.path.join(CLIP_FEATURES_PATH, file))
            if feature.shape[1] != 512:
                print(f"File {file} does not have 512 features, skipping")
                continue

            # Insert the feature into the collection
            file_name = file[:-4]
            num_frames = feature.shape[0]
            df = get_keyframe_data(MAP_KEYFRAMES_PATH, file_name)
            payloads = [{
                "pack": file_name[:3], 
                "video": file_name[4:],
                "frame_index": df.iloc[i]["frame_idx"], 
                "frame": f"00{i + 1}.jpg" if i + 1 < 10 else f"0{i + 1}.jpg" if i + 1 < 100 else f"{i + 1}.jpg"
                }
                for i in range(num_frames)
            ]

            client.upsert(
                collection_name=COLLECTION_NAME,
                points=models.Batch(
                    ids=range(start_id, start_id + feature.shape[0]),
                    vectors=feature.tolist(),
                    payloads=payloads
                )
            )
            start_id += feature.shape[0]

['L21_V001.npy', 'L21_V002.npy', 'L21_V003.npy', 'L21_V005.npy', 'L21_V006.npy']
Process L21
Process L22
Process L23
Process L24
Process L25
Process L26
Process L27
Process L28
Process L29
Process L30


## Upsert data manually (1 pack at a time)

In [ ]:
# start_id = 0

In [ ]:
# video_pack = "L30" # manually iterate from this list ["L21", "L22", "L23", ..., "L30"]

# files_per_video_pack = get_video_pack_files(CLIP_FEATURES_PATH, video_pack)
# # A list of .npy files, each file contains a list of 512 features vectors,
# # each vector is embedded from a keyframe of the video that has the same name as the file

# if len(files_per_video_pack) == 0:
#     print(f"No files found for video pack {video_pack}")
# else:
#     for file_name in files_per_video_pack:
#         feature = np.load(os.path.join(CLIP_FEATURES_PATH, file_name))
#         if feature.shape[1] != 512:
#             print(f"File {file_name} does not have 512 features, skipping")
#             continue
#         print(f"Processing file {file_name} with shape {feature.shape}")

#         # Insert the feature into the collection
#         num_frames = feature.shape[0]
#         payloads = [
#             {"origin": file_name[:-4], "frame_id": i + 1}
#             for i in range(num_frames)
#         ]

#         client.upsert(
#             collection_name=COLLECTION_NAME,
#             points=models.Batch(
#                 ids=range(start_id, start_id + feature.shape[0]),
#                 vectors=feature.tolist(),
#                 payloads=payloads
#             )
#         )
#         start_id += feature.shape[0]

## Check collection status

In [24]:
print(client.get_collection(COLLECTION_NAME))

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=177321 points_count=177321 segments_count=5 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=512, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_con

In [13]:
def count_points_without_vectors(client, collection_name):
    offset = None
    count = 0
    while True:
        points, offset = client.scroll(
            collection_name=collection_name,
            with_vectors=True,
            offset=offset,
            limit=1000  # batch size
        )
        for point in points:
            if point.vector is None:
                count += 1
        if offset is None:
            break
    return count

no_vector_count = count_points_without_vectors(client, COLLECTION_NAME)
print(f"Points with no vectors: {no_vector_count}")

Points with no vectors: 0


## Enable indexing

In [23]:
client.update_collection(
    collection_name=COLLECTION_NAME,
    optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000),
)

True

## Delete collection

In [ ]:
# client.delete_collection(collection_name=COLLECTION_NAME)

True

## Delete all points

In [ ]:
# client.delete(
#     collection_name=COLLECTION_NAME,
#     points_selector=models.FilterSelector(
#         filter=models.Filter(  # match all points
#             must=[]
#         )
#     )
# )

UpdateResult(operation_id=873, status=<UpdateStatus.COMPLETED: 'completed'>)

## Count all points

In [ ]:
client.count(
    collection_name=COLLECTION_NAME,
    count_filter=models.Filter(
        must=[]
    ),
    exact=True,
)
# 177321

CountResult(count=177321)

## Retrive a list of points

In [25]:
client.retrieve(
    collection_name=COLLECTION_NAME,
    ids=[0, 9, 99],
)

[Record(id=0, payload={'pack': 'L21', 'video': 'V001', 'frame_index': 0.0, 'frame': '001.jpg'}, vector=None, shard_key=None, order_value=None),
 Record(id=9, payload={'pack': 'L21', 'video': 'V001', 'frame_index': 1131.0, 'frame': '010.jpg'}, vector=None, shard_key=None, order_value=None),
 Record(id=99, payload={'pack': 'L21', 'video': 'V001', 'frame_index': 12450.0, 'frame': '100.jpg'}, vector=None, shard_key=None, order_value=None)]

In [ ]:
client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector, # assuming query_vector is defined, a python list of 512 features (float)
    with_payload=True,
    with_vectors=True,
    limit=20,
)

## Update payload

In [72]:
packs = ["L25", "L26"]
all_tags = dict()

In [73]:
for pack in packs:
    data = json.load(open(f"{pack}_video_tags.json", 'r'))
    videos = data.keys()
    videos = sorted(list(videos))
    for origin_video in videos:
        pack = origin_video[:3]
        video = origin_video[4:]
        client.set_payload(
            collection_name=COLLECTION_NAME,
            payload={"tags": data[origin_video]},
            points=models.Filter(
                must=[
                    models.FieldCondition(key="pack", match=models.MatchValue(value=pack)),
                    models.FieldCondition(key="video", match=models.MatchValue(value=video)),
                ]
            )
        )

    all_tags[pack] = []
    for tags in data.values():
        all_tags[pack].extend(tags)
    all_tags[pack] = sorted(list(set(all_tags[pack])))

In [74]:
hits = client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="pack", match=models.MatchValue(value="L26")),
            models.FieldCondition(key="video", match=models.MatchValue(value="V011"))
        ],
    ),
    with_payload=True,
    limit=10
)
hits

([Record(id=64704, payload={'pack': 'L26', 'video': 'V011', 'frame_index': 0.0, 'frame': '001.jpg', 'tags': ['kim chi', 'phô mai', 'lẩu']}, vector=None, shard_key=None, order_value=None),
  Record(id=64705, payload={'pack': 'L26', 'video': 'V011', 'frame_index': 21.0, 'frame': '002.jpg', 'tags': ['kim chi', 'phô mai', 'lẩu']}, vector=None, shard_key=None, order_value=None),
  Record(id=64706, payload={'pack': 'L26', 'video': 'V011', 'frame_index': 78.0, 'frame': '003.jpg', 'tags': ['kim chi', 'phô mai', 'lẩu']}, vector=None, shard_key=None, order_value=None),
  Record(id=64707, payload={'pack': 'L26', 'video': 'V011', 'frame_index': 83.0, 'frame': '004.jpg', 'tags': ['kim chi', 'phô mai', 'lẩu']}, vector=None, shard_key=None, order_value=None),
  Record(id=64708, payload={'pack': 'L26', 'video': 'V011', 'frame_index': 88.0, 'frame': '005.jpg', 'tags': ['kim chi', 'phô mai', 'lẩu']}, vector=None, shard_key=None, order_value=None),
  Record(id=64709, payload={'pack': 'L26', 'video': 'V01

In [75]:
with open("all_tags.json", 'w', encoding="utf-8") as f:
    json.dump(all_tags, f, ensure_ascii=False)